# Data Loading and Preprocessing for Austrian Parliament Speeches

Saving to csv is not optimal, there exists cell truncation in excel and some text columns are too big to be represented in excel causing spillover to next columns and all sorts of problems.

In [ ]:
import os
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


# Define standard and custom stopwords
standard_stop_words = set(stopwords.words('english'))
custom_stopwords = [
    'mr', 'mrs', 'ms', 'madam', 'honourable', 'member', 'members', 'vp', 'sp', 'today', 
    'speaker', 'deputy', 'president', 'chairman', 'chair',
    'secretary', 'lord', 'lady', 'question', 'order', 'point', 'debate',
    'motion', 'amendment', 'hear', 'minutes', 'speaking', 'close', 'abg',
    'congratulations', 'congratulate', 'thanks', 'thank', 'say', 'one', 'want',
    'know', 'think', 'believe', 'see', 'go', 'come', 'give', 'take',
    'people', 'federal', 'government', 'austria', 'austrian', 'committee', 'call', 'said',
    'already', 'committee', 'para', 'please', 'request', 'proceed', 'reading', 'people',
    'course', 'mag', 'welcome', 'council', 'open', 'written', 'contain', 'items', 'item',
    'peter', 'jonah', 'auer', 'vow', 'yes', 'no', 'following', 'orf', 'wait', 'ing', 
    'next', 'speech', 'year', 'years', 'state', 
    'also', 'would', 'like', 'may', 'must', 'upon', 'indeed', 'item',
    'session', 'meeting', 'report', 'commission', 'behalf', 'gentleman', 'gentlemen', 'ladies', 'thank',
    'applause', 'group', 'colleague', 'colleagues', 'issue', 'issues',
    'chancellor', 'court', 'ask', 'answer', 'reply', 'regard', 'regarding',
    'regards', 'respect', 'respectfully', 'sign', 'shall', 'procedure', 'declare'
]
stop_words = standard_stop_words.union(set(custom_stopwords))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Tokenize, lowercase, remove stop words, punctuation, and lemmatize."""
    text = str(text) # Ensure text is string
    tokens = word_tokenize(text.lower())
    # Keep only alphabetic tokens, remove combined stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

parent_folder = r"data folder\ParlaMint-AT-en.ana\ParlaMint-AT-en.txt"

year_folders = sorted([f for f in os.listdir(parent_folder) if f.isdigit()])

df_list = []

def process_year_folder(year_folder):
    folder_path = os.path.join(parent_folder, year_folder)
    txt_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".txt")])
    tsv_files = sorted([f for f in os.listdir(folder_path) if f.endswith("-meta.tsv")])
    
    assert len(txt_files) == len(tsv_files), f"Mismatch in TXT and TSV file counts for {year_folder}!"
    
    for txt_file, tsv_file in zip(txt_files, tsv_files):
        txt_path = os.path.join(folder_path, txt_file)
        tsv_path = os.path.join(folder_path, tsv_file)

        try:
            df_txt = pd.read_csv(txt_path, sep="\t", header=None, names=["ID", "Text"], encoding="utf-8")
            df_meta = pd.read_csv(tsv_path, sep="\t", encoding="utf-8", index_col=False)
        except Exception as e:
            print(f"Error reading files in {year_folder}: {txt_file}, {tsv_file}. Error: {e}")
            continue
        
        if len(df_txt) != len(df_meta):
            print(f"Warning Dropped row: Row count mismatch in {txt_file} ({len(df_txt)}) and {tsv_file} ({len(df_meta)}) in {year_folder}.")

        merged_df = pd.merge(df_meta, df_txt, on="ID", how="inner")
        df_list.append(merged_df)

# Process all year folders
for year in year_folders:
    process_year_folder(year)

# Combine all data into one big dataframe
AT = pd.concat(df_list, ignore_index=True)
print("\nProcessed and merged data")
print(f"Shape of the created df: {AT.shape}")

# Apply preprocessing to create the 'Processed_Text' column
print("Applying text preprocessing...")
AT['Processed_Text'] = AT['Text'].apply(preprocess_text)
print("Finished creating 'Processed_Text' column.")
AT[['Text', 'Processed_Text']].head()

Warning Dropped row: Row count mismatch in ParlaMint-AT-en_2001-07-06-021-XXI-NRSITZ-00076.txt (323) and ParlaMint-AT-en_2001-07-06-021-XXI-NRSITZ-00076-meta.tsv (325) in 2001.
Warning Dropped row: Row count mismatch in ParlaMint-AT-en_2008-09-24-023-XXIII-NRSITZ-00072.txt (486) and ParlaMint-AT-en_2008-09-24-023-XXIII-NRSITZ-00072-meta.tsv (491) in 2008.

Processed and merged data
Shape of the created df: (231752, 24)
Applying text preprocessing...
Finished creating 'Processed_Text' column.


,Text,Processed_Text
0,Please take a seat. – I also ask the photograp...,seat photographer stop activity attend first c...
1,The President of the Federal Republic of Germa...,republic germany resolution january provided a...
2,We go into the agenda and arrive at the 1st. P...,agenda arrive fishing ludmilla perfuss read fi...
3,They will be vowed unbreakable fidelity of the...,vowed unbreakable fidelity republic constant f...
4,Thank you. The Angelobung's done with it. Ladi...,angelobung done committed conscientious exerci...


In [2]:
AT.rename(columns={'Text_ID': 'Sitting_ID', 'ID': 'Speech_ID'}, inplace=True)

# drop unnecessary columns:  Body, Term, Session, Meeting, Sitting, Agenda, Subcorpus, Lang
AT.drop(columns=['Body', 'Term', 'Session', 'Meeting', 'Sitting', 'Agenda', 'Subcorpus', 'Lang'], inplace=True)

## Austria specific Data Cleaning

### Fix Party Names

In [3]:
# Fix case inconsistencies in Speaker_party
AT['Speaker_party'] = AT['Speaker_party'].replace({'Grüne': 'GRÜNE'})

party_mapping = {
        "SPÖ": "Parliamentary group of the Social Democratic Party of Austria",
        "ÖVP": "Parliamentary group of the Austrian People's Party",
        "FPÖ": "Parliamentary group of the Austrian Freedom Party",
        "GRÜNE": "Parliamentary group of The Greens - The Green Alternative",
        "BZÖ": "Austrian People's Party",
        "NEOS": "Parliamentary group of NEOS",
        "LIF": "Parliamentary group of Liberal Forum",
        "STRONACH": "Parliamentary group Team Stronach",
        "JETZT": "Parliamentary group JETZT - Pilz List",
        "-": "-" # Keep partyless designation
    }
AT['Speaker_party_name'] = AT['Speaker_party'].map(party_mapping)

print('\nOriginal Party Abbreviation Counts:')
print(AT['Speaker_party'].value_counts())


Original Party Abbreviation Counts:
Speaker_party
SPÖ         78087
ÖVP         72466
FPÖ         48411
GRÜNE       16726
BZÖ          4493
-            3855
NEOS         3565
LIF          1919
STRONACH     1612
JETZT         618
Name: count, dtype: int64


### Fix Speaker Names (Due to Name Changes)

In [4]:
# Check initial state: Count unique Speaker_name values per Speaker_ID
id_name_counts_before = AT.groupby('Speaker_ID')['Speaker_name'].nunique()
ambiguous_ids_before = id_name_counts_before[id_name_counts_before > 1]
print(f"Ambiguous Speaker IDs before cleaning: {len(ambiguous_ids_before)}")

name_mapping = {
        'Glawischnig-Piesczek, Eva': 'Glawischnig-Piesczek, Eva',
        'Glawischnig, Eva': 'Glawischnig-Piesczek, Eva',
        'Belakowitsch-Jenewein, Dagmar': 'Belakowitsch-Jenewein, Dagmar',
        'Belakowitsch, Dagmar': 'Belakowitsch-Jenewein, Dagmar',
        'Steibl, Ridi': 'Steibl, Ridi Maria',
        'Steibl, Ridi Maria': 'Steibl, Ridi Maria',
        'Bernhard, Michael': 'Bernhard-Pock, Michael',
        'Pock, Michael': 'Bernhard-Pock, Michael',
        'Fuhrmann, Silvia': 'Fuhrmann-Grünberger, Silvia',
        'Grünberger, Silvia': 'Fuhrmann-Grünberger, Silvia',
        'Holzinger-Vogtenhuber, Daniela': 'Holzinger-Vogtenhuber, Daniela',
        'Holzinger, Daniela': 'Holzinger-Vogtenhuber, Daniela',
        'Binder-Maier, Gabriele': 'Binder-Maier, Gabriele',
        'Binder, Gabriele': 'Binder-Maier, Gabriele',
        'Gartelgruber, Carmen': 'Gartelgruber-Schimanek, Carmen',
        'Schimanek, Carmen': 'Gartelgruber-Schimanek, Carmen',
        'Aumayr, Anna Elisabeth': 'Aumayr-Achatz, Anna Elisabeth',
        'Achatz, Anna Elisabeth': 'Aumayr-Achatz, Anna Elisabeth',
        'Vorderwinkler, Petra': 'Vorderwinkler-Tanzler, Petra',
        'Tanzler, Petra': 'Vorderwinkler-Tanzler, Petra',
        'Gastinger, Karin': 'Gastinger-Miklautsch, Karin',
        'Miklautsch, Karin': 'Gastinger-Miklautsch, Karin',
        'Rinner, Sylvia': 'Rinner-Prettenthaler, Sylvia',
        'Prettenthaler, Sylvia': 'Rinner-Prettenthaler, Sylvia',
        'Steßl-Mühlbacher, Sonja': 'Steßl-Mühlbacher, Sonja',
        'Steßl, Sonja': 'Steßl-Mühlbacher, Sonja'
    }

AT['Speaker_name'] = AT['Speaker_name'].replace(name_mapping)    

id_name_counts_after = AT.groupby('Speaker_ID')['Speaker_name'].nunique()
ambiguous_ids_after = id_name_counts_after[id_name_counts_after > 1]
print(f"Ambiguous Speaker IDs after cleaning: {len(ambiguous_ids_after)}")

Ambiguous Speaker IDs before cleaning: 13
Ambiguous Speaker IDs after cleaning: 0


Check for cases where the same name might correspond to different IDs (less common).

In [5]:
# Group by Speaker_name and count unique Speaker_IDs
name_id_counts = AT.groupby('Speaker_name')['Speaker_ID'].nunique()

# Filter to find names associated with multiple Speaker_IDs
ambiguous_speakers = name_id_counts[name_id_counts > 1]

print(f"\nNames associated with multiple Speaker_IDs: {len(ambiguous_speakers)}")
if len(ambiguous_speakers) > 0:
    print(ambiguous_speakers)
    ambiguous_name = ambiguous_speakers.index[0]
    print(f"\nInvestigating '{ambiguous_name}':")
    print(AT[AT['Speaker_name'] == ambiguous_name]['Speaker_ID'].value_counts())


Names associated with multiple Speaker_IDs: 1
Speaker_name
Rauch, Johannes    2
Name: Speaker_ID, dtype: int64

Investigating 'Rauch, Johannes':
Speaker_ID
PAD_21029    59
PAD_83152    45
Name: count, dtype: int64


## Filter by Word Count

### Short speeches

In [6]:
# Calculate word count for the processed text
AT['Word_Count'] = AT['Text'].apply(lambda x: len(x.split()))
    
print(f"\nShape before marking short speeches: {AT.shape}")
min_word_count = 10

# Create a new boolean column 'Is_Short_Speech'
AT['Is_Short_Speech'] = AT['Word_Count'] < min_word_count

print(f"Number of short speeches marked: {AT['Is_Short_Speech'].sum()}")

print(AT['Word_Count'].describe())


Shape before marking short speeches: (231752, 18)
Number of short speeches marked: 41119
count    231752.000000
mean        293.371669
std         461.934589
min           1.000000
25%          12.000000
50%          54.000000
75%         466.000000
max       35419.000000
Name: Word_Count, dtype: float64


### Very long speeches

In [7]:
# Calculate the 99.9th percentile for Word_Count
percentile_99_9 = AT['Word_Count'].quantile(0.999)
print(f"\n99.9th percentile for Word_Count: {percentile_99_9:.2f} words.")

# Store the original DataFrame shape
original_shape = AT.shape
print(f"Original DataFrame shape: {original_shape}")

# Filter out the top 0.1% longest speeches
AT = AT[AT['Word_Count'] <= percentile_99_9].copy()

# Display the shape of the new DataFrame
filtered_shape = AT.shape
print(f"Filtered DataFrame shape: {filtered_shape}")
print(f"Number of rows removed: {original_shape[0] - filtered_shape[0]}")

# Display basic statistics for utterance word counts on the filtered DataFrame
print("\nFiltered Utterance Word Count Statistics (top 0.1% removed):")
print(AT['Word_Count'].describe())


99.9th percentile for Word_Count: 3545.00 words.
Original DataFrame shape: (231752, 19)
Filtered DataFrame shape: (231521, 19)
Number of rows removed: 231

Filtered Utterance Word Count Statistics (top 0.1% removed):
count    231521.000000
mean        288.715672
std         430.061195
min           1.000000
25%          12.000000
50%          54.000000
75%         464.000000
max        3545.000000
Name: Word_Count, dtype: float64


The code in the cell above **does filter** the `AT` DataFrame. 
Specifically, it removes the **longest speeches**, i.e., those in the top 0.1% by word count. 
The line `AT = AT[AT['Word_Count'] <= percentile_99_9].copy()` modifies the DataFrame `AT` by keeping only the speeches whose word count is less than or equal to the 99.9th percentile.

### Merge Consecutive Speeches by Same Speaker in Same Sitting

In [8]:
print(f"\nSource AT DataFrame shape:{AT.shape}")
print(f"Number of short speeches in AT: {AT['Is_Short_Speech'].sum()}")
print(f"Number of non-short speeches in AT: {len(AT) - AT['Is_Short_Speech'].sum()}")

# Create AT_non_short by filtering out short speeches from a copy of AT
AT_non_short = AT[AT['Is_Short_Speech'] == False].copy()

# Condition for merging: same speaker AND same sitting (Text_ID)
is_prev_same_speaker_and_sitting = (
    (AT_non_short['Speaker_ID'] == AT_non_short['Speaker_ID'].shift(1)) &
    (AT_non_short['Sitting_ID'] == AT_non_short['Sitting_ID'].shift(1))
)
speech_block_id = (~is_prev_same_speaker_and_sitting).cumsum()

# Assign the block ID temporarily for grouping
AT_non_short['speech_block_id'] = speech_block_id

# Define aggregation functions
# 'Is_Short_Speech' will be 'first' for these blocks, which is False by definition of AT_non_short.
agg_funcs = {
    'Text': ' '.join,
    'Processed_Text': ' '.join
}
# Add 'first' aggregation for all other columns
first_agg_cols = {
    col: 'first' for col in AT_non_short.columns
    if col not in ['Text', 'Processed_Text', 'speech_block_id']
}
agg_funcs.update(first_agg_cols)

# Perform aggregation to create AT_preprocessed
AT_preprocessed = AT_non_short.groupby('speech_block_id').agg(agg_funcs).reset_index(drop=True)

print(f"Shape of AT_preprocessed (short speeches filtered, consecutive speeches merged): {AT_preprocessed.shape}")


Source AT DataFrame shape:(231521, 19)
Number of short speeches in AT: 41119
Number of non-short speeches in AT: 190402
Shape of AT_preprocessed (short speeches filtered, consecutive speeches merged): (170096, 19)


## Save Preprocessed Data

In [ ]:
AT.to_pickle(r'data folder\data\AT.pkl')
AT_preprocessed.to_pickle(r'data folder\data\AT_preprocessed.pkl')
print(f"Original DataFrame (AT) with shape {AT.shape} saved to 'AT.pkl'(Pickle format)")
print(f"Preprocessed DataFrame (AT_preprocessed) with shape {AT_preprocessed.shape} saved to 'AT_preprocessed.pkl'(Pickle format)")

# also save AT to .csv but without the 'Processed_Text' column
# AT.drop(columns=['Processed_Text'], inplace=True)
# AT.to_csv(r'data folder\data\AT.csv', index=False, encoding="utf-8")
# print(f"Original DataFrame (AT) with shape {AT.shape} saved to 'AT.csv' (CSV format)")

Original DataFrame (AT) with shape (231521, 19) saved to 'AT.pkl'(Pickle format)
Preprocessed DataFrame (AT_preprocessed) with shape (170096, 19) saved to 'AT_preprocessed.pkl'(Pickle format)
